##Test

In [ ]:
!pip install pytorch_lightning
!pip install segmentation-models-pytorch
!pip install torchmetrics

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm
import numpy as np
from torchmetrics import Accuracy

In [ ]:
class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return {'pic': augmented['image'][0].unsqueeze(0).float(), 'label': torch.Tensor([label]).float()}
        

In [ ]:
class LightningPredictor(nn.Module):

    def __init__(self, test_df, source_folder):
        super().__init__()

        params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)
        self.unet = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params)
        self.loss = {'Accuracy': Accuracy(0.5)}
        self.transforms = albu.Compose([albu.PadIfNeeded(480, 80),
                                        albu.RandomCrop(480, 80),
                                        albu.Resize(576, 96),
                                        ToTensorV2()])
        self.testset = SoundDataset(test_df, source_folder, self.transforms)
        self.testloader = DataLoader(self.testset, batch_size=1, shuffle=False)

    def calculate_metrics(self):

        Accuracy = list()

        with torch.no_grad():
            for batch in tqdm(self.testloader):
                pic, gt_label = batch['pic'], batch['label']
                pr_image, pr_label = self.unet(pic)
                Accuracy.append(self.loss['Accuracy'](pr_label, gt_label.type(torch.int64)).detach().numpy())

        return np.mean(Accuracy)

In [ ]:
test_df = pd.read_csv('/content/meta_val.csv') # Specify test meta
model = LightningPredictor(test_df, '/content/data') # Specify data folder
model.load_state_dict(torch.load('/content/gdrive/MyDrive/unet_22.pth')) # Specify weights
model.eval()
metrics = model.calculate_metrics()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
print('Accuracy =', metrics)

Accuracy = 0.992
